In [1]:
"""
Map creation backend file.
Created by:
Yosef Leibman
Abhishek Kapoor
Sultan Sidhu
for TD Bank Elevate Hackathon, 22 September 2018.
"""

'\nMap creation backend file.\nCreated by:\nYosef Leibman\nAbhishek Kapoor\nSultan Sidhu\nfor TD Bank Elevate Hackathon, 22 September 2018.\n'

In [2]:
import gmaps
import gmaps.datasets
import json
import random
from haversine import haversine
import geocoder
from datetime import datetime

In [3]:
gmaps.configure(api_key="<insert api key>") 

In [4]:
file = open("/Users/sultansidhu/Desktop/DataFiles/greenPParking2015.json")
stuff = file.read()
jsonstuff = json.loads(stuff)
# for x in jsonstuff['carparks']:
#     print(x, end="\n\n\n\n\n")


class ParkingClass:

    def __init__(self, location: tuple, address: str, rate_per_half_hour: float, capacity: int):
        """
        Creates a new Parking object
        """
        self.location = location
        self.address = address
        self.rate_per_halfhour = rate_per_half_hour
        self.capacity = capacity
        self.proximity_rating = 0
        self.rph_rating = 0
        self.payment_options = []
        self.occupancy_list = []
        self.user_rating = 0
        self.current_occupancy = 0

    def __str__(self) -> str:
        """
        Returns a String representation of this Parking object
        """
        g = geocoder.ip("me")
        user_loc = tuple(g.latlng)
        return "Address: {}, Rate: {}, Capacity: {}, Percent Available: {}%, Distance: {}, Payment Options: {}".format(
                                                                           self.address,
                                                                           self.rate_per_halfhour,
                                                                           self.capacity,
                                                                        (100-self.current_occupancy),
                                                                          int(get_distance(user_loc, self)),
                                                                           self.payment_options)

    def getLocationTuple(self) -> tuple:
        """
        Returns a tuple containing two floats representing the latitude and longitude of this parking
        """
        return self.location
    
    def setOccupancyList(self) -> None:
        """
        Sets the occupancy list of a ParkingLot object to a randomly generated list of occupancy percentages.
        """
        property_list = []
        while len(property_list) < 24:
            length = len(property_list)
            if length < 2:
                x = random.randint(15,23)
                property_list.append(x)
            elif length < 8:
                x = random.randint(10, 20)
                property_list.append(x)
            elif length < 10:
                x = random.randint(25, 40)
                property_list.append(x)
            elif length < 19:
                x = random.randint(45, 70)
                property_list.append(x)
            elif length < 22:
                x = random.randint(35, 55)
                property_list.append(x)
            else:
                x = random.randint(22, 32)
                property_list.append(x)
        self.occupancy_list = property_list
        
    def setPriceRating(self, rating):
        """Sets the price rating of this parking """
        self.rph_rating = rating
            
    
    def getRate(self) -> float:
        """
        Returns the rate-per-half-hour of a given parking lot.
        """
        return float(self.rate_per_halfhour)
    
    def getAddress(self):
        return self.address
    
    def getLattitude(self):
        return self.location[0]
    
    def getLongitude(self):
        return self.location[1]

    def setPaymentOptions(self, payment_option_list):
        """Sets the payment options for a specific object."""
        self.payment_options = payment_option_list
        
    def setCurrentOccupancy(self):
        """Sets the current occupancy of this object."""
        currenttime = datetime.time(datetime.now()).hour
        self.current_occupancy = self.occupancy_list[currenttime]
        
    def giveInfo(self):
        x = str(self.getAddress()) + " with rate/half-hour: $" + str(self.getRate())
        return x
        

def createParkingArray(jason):
    """Creates an array of all parking objects."""
    parkingArray = []
    for i in range(len(jason['carparks'])):
        address = jason['carparks'][i]['address']
        lattitude = jason['carparks'][i]['lat']
        longitude = jason['carparks'][i]['lng']
        rate = jason['carparks'][i]['rate_half_hour']
        capacity = jason['carparks'][i]['capacity']
        new_object = ParkingClass((float(lattitude), float(longitude)), address, rate, capacity)
        new_object.setPaymentOptions(jason['carparks'][i]['payment_options'])
        parkingArray.append(new_object)
    return parkingArray

thestuff = createParkingArray(jsonstuff)

In [5]:
marker_locations = [x.location for x in thestuff]

In [6]:
print(thestuff[0].giveInfo())

20 Charles Street with rate/half-hour: $2.0


In [7]:
park_info = [g.giveInfo() for g in thestuff]

In [8]:
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations, info_box_content=park_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
location_dict = {'lattitude': [x.getLattitude() for x in thestuff], 'longitude': [l.getLongitude() for l in thestuff]}
rate_dict = {'magnitude': [g.getRate() for g in thestuff]}

In [10]:
new_fig = gmaps.figure()
list_locations = [x.getLocationTuple() for x in thestuff]
# print(list_locations)
rates_list = [float(t.getRate())*10 for t in thestuff]
# print(rates_list)
heatmap_layer = gmaps.heatmap_layer(list_locations, weights=rate_dict['magnitude'], max_intensity=2.5, point_radius=20.0)
new_fig.add_layer(heatmap_layer)
new_fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# newnew_fig = gmaps.figure()
# list_locations = [x.getLocationTuple() for x in thestuff]
# # print(list_locations)
# rates_list = [float(t.getRate())*10 for t in thestuff]
# # print(rates_list)
# heatmap_layer = gmaps.heatmap_layer(list_locations, weights=rate_dict['magnitude'], max_intensity=2.5, point_radius=20.0)
# newnew_fig.add_layer(heatmap_layer)
# markers = gmaps.marker_layer(marker_locations, info_box_content=park_info)
# newnew_fig.add_layer(markers)
# newnew_fig
# talk this out w yosef.

In [12]:
def setAllOccupancies(lisst):
    """Returns a list of all occupancy lists set."""
    result = []
    for x in lisst:
        x.setOccupancyList()
        result.append(x)
    return result

In [13]:
def setCurrentOccupancies(lsit):
    """sets the current occupancies of all parking lot objects in the lsit."""
    for obj in lsit:
        obj.setCurrentOccupancy()

In [14]:
newlist = setAllOccupancies(thestuff)

In [15]:
setCurrentOccupancies(newlist)

In [16]:
newlist[0].current_occupancy

24

In [17]:
file = open("/Users/sultansidhu/Desktop/Python/SmartPark/new_parking_data.txt", "w")
for line in newlist:
    file.write(str(line.occupancy_list))

In [18]:
t1 = (1, 1)
t2 = (1, 3)
print(haversine(t1, t2))

222.35597879842143


In [19]:
g = geocoder.ip("me")
user_loc = tuple(g.latlng)
print(g.latlng)

[43.1923, -79.2559]


In [20]:
def get_distance(user_location, target_parking_lot):
    """Gets the distance betweent he target parking lto and the user location."""
    target_location = target_parking_lot.getLocationTuple()
    return haversine(user_location, target_location)

In [21]:
def user_distance(user_location, parking_obj_list):
    """Returns a list of distances of parking lots from the user."""
    result = []
    for object in parking_obj_list:
        result.append(get_distance(user_location, object))
    return result

In [22]:
user_distance_list = user_distance(user_loc, thestuff)

In [23]:
def setProximityRating(parking_obj_list):
    """Sets the proximity rating of each of the objects in the array."""
    g = geocoder.ip("me")
    user_loc = tuple(g.latlng)
    user_dist_list = user_distance(user_loc, parking_obj_list)
    max_distance = max(user_dist_list)
    for obj in parking_obj_list:
        obj.proximity_rating = 1-((get_distance(user_loc, obj))/max_distance)

In [24]:
list_of_prices = [x.getRate() for x in thestuff]
scores = [] 
maximum = max(list_of_prices)
for x in list_of_prices: 
    scores.append(1-(x/maximum))
    
for y in range(len(scores)):
    thestuff[y].setPriceRating(scores[y])

In [25]:
setProximityRating(thestuff)

In [26]:
def setUserRating(list_of_object):
    """Sets the user rating of the objects stored in the ParkingLot object list."""
    for obj in list_of_object:
        obj.user_rating = (1-(obj.current_occupancy/100))*(obj.proximity_rating + obj.rph_rating)/2

In [27]:
setUserRating(thestuff)

In [28]:
def display_top_15(obj_list):
    """Returns the top 15 recommendations to the user, based on rating."""
    user_rating_list = [x.user_rating for x in obj_list]
    recomms = []
    i = 0
    while i < 15:
        max_entry = max(user_rating_list)
        max_entry_index = user_rating_list.index(max_entry)
        waste = user_rating_list.pop(max_entry_index)
        recommendation = obj_list.pop(max_entry_index)
        recomms.append(recommendation)
        i += 1
    return recomms
    # the smaller the index number, more recommended it is.
    # check occupancy as well

In [29]:
recommendations = display_top_15(thestuff)

In [30]:
newnewmap = gmaps.figure()
list_locationss = [x.getLocationTuple() for x in thestuff]
# print(list_locations)
user_rating_list = [t.user_rating for t in thestuff]
# print(rates_list)
heatmapp_layer = gmaps.heatmap_layer(list_locationss, weights=user_rating_list, max_intensity=0.7, point_radius=20.0)
newnewmap.add_layer(heatmapp_layer)
newnewmap

Figure(layout=FigureLayout(height='420px'))

In [31]:
for y in recommendations:
    print(y, end="\n\n")

Address: TTC Commuter Lot - Kipling South Lot - 400 Munster Avenue, Rate: False, Capacity: 829, Percent Available: 77%, Distance: 54, Payment Options: ['Pay at Entrance Lane Stations']

Address: TTC Commuter Lot - Islington Lomond Lot - 20 Lomond Drive, Rate: False, Capacity: 283, Percent Available: 76%, Distance: 54, Payment Options: ['Pay and Display']

Address: TTC Commuter Lot - Islington Fieldway Lot - 22 Fieldway Rd, Rate: False, Capacity: 270, Percent Available: 75%, Distance: 54, Payment Options: ['Pay and Display']

Address: TTC Commuter Lot - Islington Main Lot - 3330 Bloor Street West, Rate: False, Capacity: 534, Percent Available: 75%, Distance: 54, Payment Options: ['Pay at Entrance Lane Stations']

Address: TTC Commuter Lot - Islington Cordova Lot - 70 Cordova Avenue, Rate: False, Capacity: 473, Percent Available: 74%, Distance: 55, Payment Options: ['Pay at Entrance Lane Stations']

Address: TTC Commuter Lot - Subway Crescent Lot - 95 Subway Cres, Rate: False, Capacity: 

In [32]:
fav_index = int(input("Which one of these would you like to go to?"))

Which one of these would you like to go to?11


In [33]:
def get_directions(chosen_i, recomms):
    """Returns a new map object with directions to the nearest favourite """
    chosen_station = recomms[chosen_i]
    destination = chosen_station.getLocationTuple()
    g = geocoder.ip("me")
    origin = tuple(g.latlng)
    new_map = gmaps.figure()
    directions = gmaps.directions_layer(origin, destination)
    new_map.add_layer(directions)
    return new_map 
    

In [34]:
get_directions(fav_index, recommendations)

Figure(layout=FigureLayout(height='420px'))